In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import folium
from folium import Choropleth

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

from sklearn.ensemble import RandomForestRegressor

In [3]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('../bin/data_preprocess.csv')

In [4]:
df[df['id_sobrealimentacion'] == 'Tipo de sobrealimentador']

,id_provincia,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,deposito,combustible,cilindrada,tipo_cambio,id_traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo


In [5]:
stats = []
for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    stats_df = pd.DataFrame(stats, columns=['Columna', 'Valores únicos', '% nulos', '% of values in the biggest category', 'tipo'])
stats_df.sort_values('% nulos', ascending=False)

,Columna,Valores únicos,% nulos,% of values in the biggest category,tipo
31,co2,234,57.653690,57.653690,float64
16,consumo_carretera,91,53.922259,53.922259,float64
17,consumo_urbano,166,53.913452,53.913452,float64
15,consumo_medio,131,48.951911,48.951911,float64
33,id_sobrealimentacion,4,43.009806,43.009806,object
18,deposito,86,35.535201,35.535201,float64
32,num_cilindros,8,35.232811,35.232811,float64
20,cilindrada,233,34.225823,34.225823,float64
27,velocidad_max,156,30.159709,30.159709,float64
10,capacidad_maletero,393,30.001174,30.001174,float64


In [6]:
df.sample(10)

,id_provincia,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,deposito,combustible,cilindrada,tipo_cambio,id_traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo
21169,Barcelona,87784.0,AUTOHERO BARCELONA,C,12.0,11899.0,27765.0,4360.0,1834.0,1520.0,380.0,5.0,2670.0,1735.0,5.0,4.6,3.2,5.9,60.0,Gasolina,1199.0,Manual,delantera,6.0,96.0,130.0,230.0,210.0,10.8,7,2017,104.0,3.0,NaN,CITROEN,C4
7856,Madrid,31722.0,"AUTO ELIA, Volvo (SEDE PRINCIPAL)",0 EMISIONES,24.0,39900.0,48647.0,4425.0,1863.0,1652.0,445.0,5.0,2702.0,2460.0,5.0,NaN,NaN,NaN,NaN,Eléctrico,NaN,Automático,total permanente,1.0,170.0,231.0,NaN,NaN,4.9,5,2022,NaN,NaN,NaN,VOLVO,XC40
15854,Madrid,62750.0,GRUPO AMARAUTO,C,12.0,21490.0,36150.0,4500.0,1969.0,1654.0,580.0,5.0,2730.0,1960.0,5.0,5.1,4.9,7.0,53.0,Gasolina,1199.0,Automático,delantera,8.0,96.0,130.0,230.0,188.0,10.3,3,2021,116.0,3.0,Turbo,CITROEN,C5 AIRCROSS
13754,Madrid,53767.0,CLICARS MADRID,ECO,12.0,15990.0,22800.0,4228.0,1797.0,1566.0,536.0,5.0,2639.0,1741.0,5.0,6.6,5.6,8.1,48.0,Gasolina/gas,NaN,Manual,delantera,5.0,74.0,100.0,170.0,173.0,13.3,5,2021,107.0,3.0,Turbo,RENAULT,CAPTUR
1115,Alicante,156000.0,SLS COCHES.,C,12.0,6999.0,18350.0,3969.0,1722.0,1468.0,276.0,5.0,2489.0,1575.0,5.0,3.6,3.3,4.4,42.0,Diesel,1560.0,Manual,delantera,5.0,70.0,95.0,205.0,181.0,11.7,4,2014,95.0,4.0,Turbo,FORD,FIESTA
28640,Murcia,12280.0,AUDI HUERTAS MOTOR MURCIA,C,12.0,24000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,Gasolina,NaN,Manual,NaN,NaN,NaN,110.0,NaN,NaN,NaN,8,2023,NaN,NaN,NaN,AUDI,A1
27386,Madrid,103969.0,CLICARS MADRID,C,12.0,19990.0,35375.0,4515.0,1865.0,1650.0,591.0,5.0,2680.0,2085.0,5.0,NaN,NaN,NaN,54.0,Gasolina,1598.0,Manual,delantera,6.0,112.0,150.0,250.0,189.0,10.3,9,2022,NaN,4.0,Turbo,KIA,SPORTAGE
8955,Madrid,204500.0,Clidrive Group,C,12.0,8190.0,16190.0,4062.0,1732.0,1448.0,300.0,5.0,2589.0,1563.0,5.0,4.7,4.1,5.7,45.0,Gasolina,NaN,Manual,delantera,5.0,66.0,90.0,140.0,167.0,12.2,11,2016,105.0,3.0,Turbo,RENAULT,CLIO
3718,Murcia,19749.0,AUDI HUERTAS MOTOR MURCIA,NaN,12.0,44500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,Diesel,NaN,Automático,NaN,NaN,NaN,163.0,NaN,NaN,NaN,6,2023,NaN,NaN,NaN,AUDI,Q5
25654,Navarra,7370.0,FLEXICAR NAVARRA,C,12.0,25490.0,32240.0,4610.0,1876.0,1685.0,448.0,5.0,2720.0,NaN,5.0,NaN,NaN,NaN,55.0,Gasolina,1490.0,Automático,delantera,7.0,119.0,162.0,250.0,190.0,9.9,1,2024,NaN,4.0,Turbo,MG,HS


In [7]:
listado_provincias = {}
index = 1
for detalle in df['id_provincia'].unique():
    if pd.notnull(detalle): 
        listado_provincias[index] = detalle
        index += 1

df_provincias = pd.DataFrame.from_dict(listado_provincias, orient='index', columns=['nombre_provincia']).reset_index()
df_provincias.rename(columns={'index': 'id_provincia'}, inplace=True)
df_provincias.to_csv('../bin/listado_provincias.csv', index=False)

conversion_provincias = {v: k for k, v in listado_provincias.items()}
df['id_provincia'] = df['id_provincia'].map(conversion_provincias)

In [8]:
listado_concesionarios = {}
index = 1
for detalle in df['id_concesionario'].unique():
    if pd.notnull(detalle): 
        listado_concesionarios[index] = detalle
        index += 1

df_concesionarios = pd.DataFrame.from_dict(listado_concesionarios, orient='index', columns=['nombre_concesionario']).reset_index()
df_concesionarios.rename(columns={'index': 'id_concesionario'}, inplace=True)
df_concesionarios.to_csv('../bin/listado_concesionarios.csv', index=False)

conversion = {v: k for k, v in listado_concesionarios.items()}
df['id_concesionario'] = df['id_concesionario'].map(conversion)

In [9]:
listado_modelos = {}
index = 1
for detalle in df['id_modelo'].unique():
    if pd.notnull(detalle): 
        listado_modelos[index] = detalle
        index += 1

df_modelos = pd.DataFrame.from_dict(listado_modelos, orient='index', columns=['nombre_modelo']).reset_index()
df_modelos.rename(columns={'index': 'id_modelo'}, inplace=True)
df_modelos.to_csv('../bin/listado_modelos.csv', index=False)

conversion = {v: k for k, v in listado_modelos.items()}
df['id_modelo'] = df['id_modelo'].map(conversion)

In [10]:
marca_df = pd.read_csv('../bin/listado_marcas.csv')

marca_df['nombre_marca'] = marca_df['nombre_marca'].str.upper()
listado_marcas = marca_df.set_index('id_marca')['nombre_marca'].to_dict()

conversion = {v: k for k, v in listado_marcas.items()}
df['id_marca'] = df['id_marca'].map(conversion)

In [11]:
listado_distintivo_ambiental = {}
index = 1
for detalle in df['id_distintivo_ambiental'].unique():
    if pd.notnull(detalle): 
        listado_distintivo_ambiental[index] = detalle
        index += 1
    
df_distintivo_ambiental = pd.DataFrame.from_dict(listado_distintivo_ambiental, orient='index', columns=['nombre_distintivo']).reset_index()
df_distintivo_ambiental.rename(columns={'index': 'id_distintivo'}, inplace=True)
df_distintivo_ambiental.to_csv('../bin/listado_distintivo_ambiental.csv', index=False)

conversion = {v: k for k, v in listado_distintivo_ambiental.items()}
df['id_distintivo_ambiental'] = df['id_distintivo_ambiental'].map(conversion)

In [12]:
listado_combustibles = {}
index = 1
for detalle in df['combustible'].unique():
    if pd.notnull(detalle): 
        listado_combustibles[index] = detalle
        index += 1
    
df_combustibles = pd.DataFrame.from_dict(listado_combustibles, orient='index', columns=['nombre_combustible']).reset_index()
df_combustibles.rename(columns={'index': 'id_combustible'}, inplace=True)
df_combustibles.to_csv('../bin/listado_combustibles.csv', index=False)

conversion = {v: k for k, v in listado_combustibles.items()}
df['id_combustible'] = df['combustible'].map(conversion)

df = df.drop(columns=['combustible'])

In [13]:
listado_tipos_cambio = {}
index = 1
for detalle in df['tipo_cambio'].unique():
    if pd.notnull(detalle): 
        listado_tipos_cambio[index] = detalle
        index += 1
    
df_tipos_cambio = pd.DataFrame.from_dict(listado_tipos_cambio, orient='index', columns=['nombre_tipo_cambio']).reset_index()
df_tipos_cambio.rename(columns={'index': 'id_tipo_cambio'}, inplace=True)
df_tipos_cambio.to_csv('../bin/listado_tipos_cambio.csv', index=False)

tipos_cambio_to_indices = {v: k for k, v in listado_tipos_cambio.items()}
df['id_tipo_cambio'] = df['tipo_cambio'].map(tipos_cambio_to_indices)

df = df.drop(columns=['tipo_cambio'])

In [14]:
listado_sobrealimentacion = {}
index = 1
for detalle in df['id_sobrealimentacion'].unique():
    if pd.notnull(detalle): 
        listado_sobrealimentacion[index] = detalle
        index += 1
    
df_sobrealimentacion = pd.DataFrame.from_dict(listado_sobrealimentacion, orient='index', columns=['nombre_sobrealimentacion']).reset_index()
df_sobrealimentacion.rename(columns={'index': 'id_sobrealimentacion'}, inplace=True)
df_sobrealimentacion.to_csv('../bin/listado_sobrealimentacion.csv', index=False)

conversion = {v: k for k, v in listado_sobrealimentacion.items()}
df['id_sobrealimentacion'] = df['id_sobrealimentacion'].map(conversion)

In [15]:
listado_traccion = {}
index = 1
for detalle in df['id_traccion'].unique():
    if pd.notnull(detalle): 
        listado_traccion[index] = detalle
        index += 1
    
df_traccion = pd.DataFrame.from_dict(listado_traccion, orient='index', columns=['nombre_traccion']).reset_index()
df_traccion.rename(columns={'index': 'id_traccion'}, inplace=True)
df_traccion.to_csv('../bin/listado_traccion.csv', index=False)

conversion = {v: k for k, v in listado_traccion.items()}
df['id_traccion'] = df['id_traccion'].map(conversion)

In [16]:
df.head()

,id_provincia,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,deposito,cilindrada,id_traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo,id_combustible,id_tipo_cambio
0,1.0,58000.0,1,1.0,12.0,19990.0,37250.0,4633.0,1811.0,1429.0,480.0,5.0,2810.0,2045.0,4.0,4.5,4.0,5.3,57.0,1995.0,1.0,6.0,110.0,150.0,320.0,215.0,8.7,11,2018,118.0,4.0,1.0,8.0,1.0,1.0,1
1,1.0,34680.0,1,2.0,12.0,12990.0,19300.0,3571.0,1627.0,1488.0,185.0,4.0,2300.0,1055.0,3.0,NaN,NaN,NaN,35.0,NaN,2.0,6.0,52.0,70.0,92.0,167.0,13.8,12,2021,NaN,3.0,2.0,19.0,2.0,2.0,1
2,1.0,66933.0,2,1.0,12.0,17390.0,26650.0,4377.0,1806.0,1590.0,430.0,5.0,2646.0,1910.0,5.0,3.8,3.6,4.2,55.0,1461.0,2.0,6.0,81.0,110.0,260.0,182.0,11.9,4,2017,103.0,4.0,2.0,50.0,3.0,1.0,1
3,2.0,159000.0,3,1.0,12.0,8000.0,22425.0,4702.0,1809.0,1499.0,540.0,5.0,2662.0,1985.0,5.0,3.6,3.3,4.2,48.0,1598.0,2.0,6.0,81.0,110.0,300.0,195.0,11.4,1,2017,99.0,4.0,2.0,52.0,4.0,1.0,1
4,1.0,62695.0,4,3.0,12.0,22990.0,44700.0,4545.0,1805.0,1685.0,404.0,5.0,2670.0,2425.0,5.0,NaN,NaN,NaN,43.0,2360.0,3.0,1.0,138.0,188.0,193.0,162.0,10.9,7,2022,NaN,4.0,2.0,48.0,5.0,3.0,2


In [17]:
df.to_csv('../bin/prestaciones.csv', index=False)